In [2]:
from src.utils.database import Database

In [66]:
import pandas as pd
import random

def random_unique_integers(start, end, count):
    if count > (end - start + 1):
        raise ValueError("Count must be less than or equal to the range size.")
    
    return random.sample(range(start, end + 1), count)

output = []
with Database() as db:
    users = db._run_query("""
        SELECT
            id, username, designated_corpus_id
        FROM 
            users 
        WHERE 
            designated_corpus_id is not null
    """)
    for _, user in users.iterrows():
        last_audios = db._run_query(f"""
            SELECT
                *
            FROM 
                Audio
            WHERE
                responsible_user_id = {user.id}
            ORDER BY
                id DESC
            LIMIT
                3
        """)
        if not last_audios.empty:
            max_segment_num = None
            i = 0
            while max_segment_num is None and i != 3 :
                total_segments_transcribed = db._run_query(f"""
                    SELECT
                        MAX(segment_num) as max_segment
                    FROM 
                        Dataset
                    WHERE
                        audio_id = {last_audios.id[i]} and text is not null
                """)
                max_segment_num = total_segments_transcribed.max_segment[0]
                i += 1
            segment_numbers = random_unique_integers(1, max_segment_num, 5)

            
            # Join the list of integers as a comma-separated string
            id_list_str = ", ".join(map(str, segment_numbers))

            # Generating the SQL query
            sql_query = f"SELECT * FROM Dataset WHERE audio_id = {last_audios.id[0]} and segment_num IN ({id_list_str}) and text != '###'"
            
            segments = db._run_query(sql_query)
            for _, segment in segments.iterrows():
                output.append({
                    "username": user.username,
                    "corpus": "mupe" if user.designated_corpus_id == 1 else "nurc",
                    "audio": last_audios.name[i-1],
                    "segment_num": f"{segment.segment_num}/{max_segment_num}",
                    "duration": segment.duration,
                    "text_asr": segment.text_asr,
                    "transcription": segment.text,
                    "link": "http://143.107.183.175:14888/static/Dataset/" + segment.file_path
                })

In [67]:
output_df = pd.DataFrame(output)

In [68]:
output_df.to_excel("Segmentos para análise sem inválidos.xlsx", index=False)